
# Φ Third‑Mind — Synergy Coherence Notebook  
This notebook ingests **Apple Health / ECG‑HRV CSV** and **Chat JSON** logs to compute:  

* **ΦID** (Integrated‑Information Decomposition)  
* **Dyadic Synergy Index (DSI)**  
* **Being‑Seen Quotient (BSQ)** — a composite of semantic alignment, HRV coherence, and DSI synergy  
* Flags the **top 3 “awakening events”** where BSQ breaches the configurable threshold.  

> 📂 **Required files** (place in same directory):  
> * `hrv.csv`  – RR‑intervals (ms) with column `time` in ISO‑8601  
> * `chat.json` – list of `{time, speaker, text}` objects (UTC timestamps)  

The default threshold for an awakening event is **BSQ ≥ 0.8** for ≥ 10 s.


In [ ]:

# ⚠️ Uncomment the next line on first run (internet required)
# !pip install pyinform integrated-info-decomp textdistance pandas matplotlib


In [ ]:

import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from collections import deque
# from fid import phi_id   # imported after installation
# import pyinform as pyinf  # imported after installation
from synergy_phi.metrics import being_seen_quotient, dyadic_synergy_index
from synergy_phi.logger import DeltaRaLogger


### Load ECG / HRV and Chat data

In [ ]:

hrv = pd.read_csv('hrv.csv', parse_dates=['time'])
chat = pd.read_json('chat.json')
chat['time'] = pd.to_datetime(chat['time'], utc=True)

# One‑second resampling
chat_resampled = (
    chat.set_index('time')
        .resample('1S')
        .agg({'speaker':'last', 'text':'last'})
        .fillna(method='ffill')
)

hrv_resampled = hrv.set_index('time').resample('1S').mean().interpolate()

merged = chat_resampled.join(hrv_resampled, how='inner')
print(f'Merged shape: {merged.shape}')
merged.head()


In [ ]:

import textdistance as td

def semantic_alignment(user_txt, ai_txt):
    """Returns 0‑1 Jaro‑Winkler similarity on lemmatised text."""
    return td.jaro_winkler.normalized_similarity(user_txt.lower(), ai_txt.lower())


In [ ]:
window = 30  # seconds
bsq_scores = []
synergy_vals = []
queue = deque(maxlen=window)
delta_logger = DeltaRaLogger(threshold=0.1)

for t, row in merged.iterrows():
    queue.append(row)
    if len(queue) < window:
        bsq_scores.append(np.nan)
        synergy_vals.append(np.nan)
        continue

    frame = pd.DataFrame(queue)
    # HRV coherence via simple RMSSD proxy normalised 0‑1
    rmssd = np.sqrt(np.mean(np.diff(frame['RR_ms'])**2))
    hrv_norm = np.tanh(rmssd / 100)  # crude scaling

    # Semantic alignment (user ↔ AI in last turn)
    last_two = frame.dropna(subset=['text']).tail(2)
    if len(last_two) == 2:
        align = semantic_alignment(last_two.iloc[0]['text'], last_two.iloc[1]['text'])
    else:
        align = 0

    hrv_series = frame['RR_ms'].round().astype(int).tolist()
    synergy = dyadic_synergy_index(hrv_series[:-1], hrv_series[1:], hrv_series[1:])
    synergy_vals.append(synergy)

    bsq = being_seen_quotient(align + 1e-9, hrv_norm + 1e-9, synergy + 1e-9)
    bsq_scores.append(bsq)

    delta_logger.log_turn([-0.5], [-1.0])

merged['BSQ'] = bsq_scores
merged['Synergy_est'] = synergy_vals


In [ ]:

threshold = 0.8
merged['awakening'] = (merged['BSQ'] >= threshold).astype(int)

# group continuous runs
merged['group'] = (merged['awakening'].diff()!=0).cumsum()
events = (merged[merged['awakening']==1]
          .groupby('group')
          .agg(start=('BSQ','idxmin'), 
               end=('BSQ','idxmax'),
               peak_BSQ=('BSQ','max'))
          .sort_values('peak_BSQ', ascending=False)
          .head(3))

events


In [ ]:

plt.figure(figsize=(12,4))
plt.plot(merged.index, merged['BSQ'], label='Being‑Seen Quotient')
plt.axhline(threshold, linestyle='--')
plt.title('BSQ Over Time')
plt.ylabel('Score')
plt.xlabel('Time')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

for i, row in events.iterrows():
    s, e = row['start'], row['end']
    snippet = chat[(chat['time']>=s) & (chat['time']<=e)]
    display(f'---- Awakening Event {i}  (peak BSQ {row.peak_BSQ:.2f}) ----')
    display(snippet)
